In [1]:
from manim import *

from rc_lib import math_types as T
from rc_lib import style
from rc_lib.math_utils import geometry
from rc_lib.design_utils import plate, sketch
from rc_lib.view_utils import title_sequence

quality = "ql"

Manim Community v0.17.2

In [2]:
inner_color = style.Palette.GREEN
boundary_color = style.Palette.BLUE
factory = plate.PlateCircleFactory()
factory.set_inner_color(inner_color).set_outer_color(boundary_color)

title = title_sequence.TitleSequence(default_color=boundary_color)

# TODO:
# plategroup factory
# basic website
# trim

The reccomended way to create plates is by first drawing your holes and other relevant geometry, marking out the outer boundary using a second set of circles, and then connecting the boundary with tangent lines.

In [3]:
%%manim -v WARNING --disable_caching -$quality IntakePlateScene

class IntakePlateScene(Scene):
    def setup(self):
        small_base = factory.make_generator(0.15, 0.2)
        medium_base = factory.make_generator(0.4, 0.2)

        front_hole = geometry.point_2d(-4, -3)
        middle_hole = geometry.point_2d(-1.5, 0.25)
        back_hole = geometry.point_2d(2.5, 1.5)
        back_offset = geometry.point_2d(0.8, 0.75)

        points = [
            medium_base(front_hole),
            medium_base(middle_hole),
            medium_base(back_hole),
            small_base(back_hole + back_offset),
            small_base(back_hole + geometry.point_2d(1, -0.2)),
            small_base((middle_hole + back_hole) / 2),
            small_base((front_hole + middle_hole) / 2)
        ]
        boundary_order = [1, 3, 4, 0]
        self._plate_group = plate.PlateGroup(points, boundary_order, boundary_color=boundary_color)
        title.reset()

    def construct(self):
        self.play(title.next("Draw plate holes", color=inner_color))
        self.play(self._plate_group.draw_inner_circles(lag_ratio=0.5))
        self.wait()

        self.play(title.next("Add larger circles"))
        self.play(self._plate_group.draw_outer_circles(lag_ratio=0.5))
        self.wait()

        self.play(title.next("Connect boundary"))
        self.play(self._plate_group.draw_boundary(lag_ratio=0.75, run_time=5))
        # self.wait()

        # self.play(title.next("Trim", color=boundary_color))
        # self.play(plate_group.trim(), run_time=5)

        self.wait(style.END_DELAY)

In the event a hole is too close to the edge of a plate, you may need to redraw the boundary.

In [4]:
%%manim -v WARNING --disable_caching -$quality BoundaryRedrawScene

class BoundaryRedrawScene(Scene):
    def setup(self):
        generator = factory.make_generator(1.75, 0.75)
        self._left = generator(geometry.point_2d(-6, -2))
        self._right = generator(geometry.point_2d(6, -2))
        self._middle = factory.make(1, 0.75, geometry.point_2d(0, -0.75))

        self._line = plate.PlateCircle.tangent_line(self._left, self._right, style.Palette.RED)
        self.add(self._left, self._right, self._line, self._middle.inner_circle())

        title.reset()

    def construct(self):
        self.play(title.next("Add outer circle"))
        self.play(self._middle.draw_outer_circle())

        self.play(title.next("Redraw boundary"))
        self.play(Uncreate(self._line))
        self.wait(0.25)
        self.play(Create(plate.PlateCircle.tangent_line(self._left, self._middle, boundary_color)))
        self.play(Create(plate.PlateCircle.tangent_line(self._middle, self._right, boundary_color)))

        self.wait(style.END_DELAY)

In [5]:
%%manim -v WARNING -$quality LineConstraintScene
import operator
from rc_lib.design_utils.sketch import LineEnd

class LineConstraintScene(Scene):
    def setup(self):
        generator = factory.make_generator(1.75, 0.75)
        self._left = generator(geometry.point_2d(-6, -2))
        self._right = generator(geometry.point_2d(6, -2))
        self.add(self._left, self._right)

        self._tangent_points = plate.PlateCircle.tangent_points(self._left, self._right)

        left_start_point = self._tangent_points[0] + geometry.point_2d(1.75, 0.75)
        right_start_point = self._tangent_points[1] + geometry.point_2d(-2, 0.5)
        self._line = sketch.SketchLine(left_start_point, right_start_point, color=boundary_color)

        title.reset()
    
    def get_vars(self, line_end: LineEnd, *keys: str):
        return [self._get_var(line_end, key) for key in keys]

    def _get_var(self, line_end: LineEnd, key: str):
        if (key == "tangent_point"):
            return self._tangent_points[line_end]

        attributes = {
            "circle" : ("_left", "_right"),
            "tip" : ("_line.start", "_line.end"),
            "point" : ("_line.start_point", "_line.end_point"),
        }
        result = operator.attrgetter(attributes[key][line_end])(self)
        return result() if key == "point" or key == "tip" else result

    def construct(self):
        self.play(title.next("Create line"))
        self.play(self._line.create())

        self.play(title.next("Add coincident constraints"))
        self.do_coincident_move(LineEnd.START)
        self.do_coincident_move(LineEnd.END)

        self.play(title.next("Add tangent constraints"))
        self.do_tangent_move(LineEnd.START)
        self.do_tangent_move(LineEnd.END)

        self.wait(style.END_DELAY)
    
    def do_coincident_move(self, line_end: LineEnd):
        self._do_flash(line_end)
        new_point = self._coincident_point(line_end)
        self.play(Transform(self._line, self._line.copy().set_position(new_point, line_end)))

    def _coincident_point(self, line_end: LineEnd) -> T.Point2d:
        circle, point = self.get_vars(line_end, "circle", "point")
        return circle.center() + geometry.normalize(point - circle.center()) * circle.outer_radius()
    
    def do_tangent_move(self, line_end: LineEnd):
        self._do_flash(line_end)
        circle, tangent_point = self.get_vars(line_end, "circle", "tangent_point")
        angle = self._tangent_angle(line_end)
        self.play(Transform(self._line, self._line.copy().set_position(tangent_point, line_end),
            path_arc=angle, part_arg_centers=[circle.center()]))

    def _tangent_angle(self, line_end: LineEnd) -> float:
        point, tangent_point, circle = self.get_vars(line_end, "point", "tangent_point", "circle")
        return (1 if line_end == LineEnd.START else -1) * angle_between_vectors(point - circle.center(), tangent_point - circle.center())
    
    def _do_flash(self, line_end: LineEnd):
        circle, tip = self.get_vars(line_end, "circle", "tip")
        self.play(Flash(tip, run_time=0.75))
        self.play(Flash(circle, flash_radius=circle.outer_radius(), num_lines=40, run_time=0.75))